## مقدمه

این درس شامل موارد زیر خواهد بود:  
- فراخوانی تابع چیست و کاربردهای آن  
- چگونه یک فراخوانی تابع با استفاده از OpenAI ایجاد کنیم  
- چگونه یک فراخوانی تابع را در یک برنامه ادغام کنیم  

## اهداف یادگیری

پس از اتمام این درس، شما خواهید دانست و درک خواهید کرد:  

- هدف استفاده از فراخوانی تابع  
- راه‌اندازی فراخوانی تابع با استفاده از سرویس OpenAI  
- طراحی فراخوانی‌های تابع مؤثر برای کاربرد برنامه‌های خود


## درک فراخوانی توابع

برای این درس، می‌خواهیم ویژگی‌ای برای استارتاپ آموزشی خود بسازیم که به کاربران اجازه می‌دهد از یک چت‌بات برای پیدا کردن دوره‌های فنی استفاده کنند. ما دوره‌هایی را پیشنهاد خواهیم داد که با سطح مهارت، نقش فعلی و فناوری مورد علاقه آن‌ها مطابقت دارد.

برای تکمیل این کار از ترکیبی از موارد زیر استفاده خواهیم کرد:
 - `OpenAI` برای ایجاد تجربه چت برای کاربر
 - `Microsoft Learn Catalog API` برای کمک به کاربران در پیدا کردن دوره‌ها بر اساس درخواست کاربر
 - `Function Calling` برای گرفتن پرسش کاربر و ارسال آن به یک تابع برای انجام درخواست API.

برای شروع، بیایید ببینیم چرا در وهله اول می‌خواهیم از فراخوانی تابع استفاده کنیم:

print("پیام‌ها در درخواست بعدی:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # دریافت پاسخ جدید از GPT که می‌تواند پاسخ تابع را ببیند


print(second_response.choices[0].message)


### چرا فراخوانی تابع

اگر هر درس دیگری را در این دوره گذرانده باشید، احتمالاً قدرت استفاده از مدل‌های زبان بزرگ (LLMها) را درک کرده‌اید. امیدواریم بتوانید برخی از محدودیت‌های آن‌ها را نیز ببینید.

فراخوانی تابع ویژگی‌ای از سرویس OpenAI است که برای حل چالش‌های زیر طراحی شده است:

قالب‌بندی نامنظم پاسخ‌ها:
- قبل از فراخوانی تابع، پاسخ‌های مدل زبان بزرگ ساختارمند و منسجم نبودند. توسعه‌دهندگان مجبور بودند کدهای اعتبارسنجی پیچیده‌ای بنویسند تا هر نوع تغییر در خروجی را مدیریت کنند.

ادغام محدود با داده‌های خارجی:
- پیش از این ویژگی، وارد کردن داده‌ها از بخش‌های دیگر یک برنامه به زمینه چت دشوار بود.

با استانداردسازی قالب‌های پاسخ و امکان ادغام بی‌وقفه با داده‌های خارجی، فراخوانی تابع توسعه را ساده‌تر کرده و نیاز به منطق اعتبارسنجی اضافی را کاهش می‌دهد.

کاربران نمی‌توانستند پاسخ‌هایی مانند «وضعیت آب و هوای فعلی در استکهلم چیست؟» دریافت کنند. این به این دلیل بود که مدل‌ها محدود به زمانی بودند که داده‌ها آموزش دیده بودند.

بیایید به مثال زیر نگاه کنیم که این مشکل را نشان می‌دهد:

فرض کنیم می‌خواهیم یک پایگاه داده از اطلاعات دانش‌آموزان ایجاد کنیم تا بتوانیم دوره مناسب را به آن‌ها پیشنهاد دهیم. در زیر دو توصیف از دانش‌آموزان داریم که در داده‌هایشان بسیار مشابه هستند.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

ما می‌خواهیم این را به یک مدل زبان بزرگ (LLM) ارسال کنیم تا داده‌ها را تجزیه کند. این بعداً می‌تواند در برنامه ما برای ارسال به یک API یا ذخیره در یک پایگاه داده استفاده شود.

بیایید دو درخواست یکسان ایجاد کنیم که به مدل زبان بزرگ دستور می‌دهیم به چه اطلاعاتی علاقه‌مند هستیم:


ما می‌خواهیم این را به یک مدل زبان بزرگ (LLM) ارسال کنیم تا بخش‌های مهم برای محصول ما را تجزیه و تحلیل کند. بنابراین می‌توانیم دو درخواست یکسان برای راهنمایی مدل زبان بزرگ ایجاد کنیم:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


پس از ایجاد این دو پرامپت، آن‌ها را با استفاده از `openai.ChatCompletion` به مدل زبان بزرگ ارسال می‌کنیم. ما پرامپت را در متغیر `messages` ذخیره می‌کنیم و نقش را به `user` اختصاص می‌دهیم. این کار برای شبیه‌سازی پیامی است که توسط یک کاربر به یک چت‌بات نوشته می‌شود.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

اکنون می‌توانیم هر دو درخواست را به مدل زبان بزرگ ارسال کنیم و پاسخ دریافتی را بررسی کنیم.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

با اینکه پرامپت‌ها یکسان هستند و توضیحات مشابه‌اند، می‌توانیم فرمت‌های مختلفی از ویژگی `Grades` دریافت کنیم.

اگر سلول بالا را چندین بار اجرا کنید، فرمت می‌تواند به صورت `3.7` یا `3.7 GPA` باشد.

این به این دلیل است که مدل زبان بزرگ (LLM) داده‌های بدون ساختار را به صورت پرامپت نوشته شده دریافت می‌کند و همچنین داده‌های بدون ساختار بازمی‌گرداند. ما نیاز داریم فرمت ساختاری داشته باشیم تا بدانیم هنگام ذخیره یا استفاده از این داده‌ها چه انتظاری باید داشته باشیم.

با استفاده از فراخوانی تابع، می‌توانیم مطمئن شویم که داده‌های ساختاریافته دریافت می‌کنیم. هنگام استفاده از فراخوانی تابع، مدل زبان بزرگ در واقع هیچ تابعی را فراخوانی یا اجرا نمی‌کند. در عوض، ما ساختاری برای مدل ایجاد می‌کنیم تا در پاسخ‌های خود از آن پیروی کند. سپس از آن پاسخ‌های ساختاریافته استفاده می‌کنیم تا بدانیم در برنامه‌های خود چه تابعی را اجرا کنیم.


![نمودار جریان فراخوانی تابع](../../../../translated_images/fa/Function-Flow.083875364af4f4bb.webp)


سپس می‌توانیم آنچه از تابع بازگردانده شده است را گرفته و به مدل زبان بزرگ (LLM) ارسال کنیم. مدل زبان بزرگ سپس با استفاده از زبان طبیعی به پرسش کاربر پاسخ خواهد داد.


### موارد استفاده از فراخوانی توابع

**فراخوانی ابزارهای خارجی**  
چت‌بات‌ها در ارائه پاسخ به سوالات کاربران بسیار خوب عمل می‌کنند. با استفاده از فراخوانی توابع، چت‌بات‌ها می‌توانند از پیام‌های کاربران برای انجام وظایف خاص استفاده کنند. به عنوان مثال، یک دانش‌آموز می‌تواند از چت‌بات بخواهد «ارسال ایمیل به استاد من با این مضمون که به کمک بیشتری در این موضوع نیاز دارم». این می‌تواند یک فراخوانی تابع به `send_email(to: string, body: string)` باشد.

**ایجاد پرس‌وجوهای API یا پایگاه داده**  
کاربران می‌توانند با استفاده از زبان طبیعی اطلاعاتی را پیدا کنند که به یک پرس‌وجوی قالب‌بندی شده یا درخواست API تبدیل می‌شود. مثالی از این مورد می‌تواند معلمی باشد که درخواست می‌کند «دانش‌آموزانی که آخرین تکلیف را انجام داده‌اند چه کسانی هستند» که می‌تواند تابعی به نام `get_completed(student_name: string, assignment: int, current_status: string)` را فراخوانی کند.

**ایجاد داده‌های ساختاریافته**  
کاربران می‌توانند یک بلوک متن یا CSV را گرفته و از LLM برای استخراج اطلاعات مهم از آن استفاده کنند. به عنوان مثال، یک دانش‌آموز می‌تواند یک مقاله ویکی‌پدیا درباره توافقات صلح را به کارت‌های فلش هوش مصنوعی تبدیل کند. این کار می‌تواند با استفاده از تابعی به نام `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` انجام شود.


## 2. ایجاد اولین فراخوانی تابع شما

فرآیند ایجاد یک فراخوانی تابع شامل ۳ مرحله اصلی است:  
1. فراخوانی API تکمیل چت با فهرستی از توابع شما و یک پیام کاربر  
2. خواندن پاسخ مدل برای انجام یک عمل، یعنی اجرای یک تابع یا فراخوانی API  
3. انجام یک فراخوانی دیگر به API تکمیل چت با پاسخ از تابع شما برای استفاده از آن اطلاعات جهت ایجاد پاسخ به کاربر.


![جریان یک فراخوانی تابع](../../../../translated_images/fa/LLM-Flow.3285ed8caf4796d7.webp)


### عناصر یک فراخوانی تابع

#### ورودی کاربران

اولین مرحله ایجاد یک پیام کاربر است. این می‌تواند به صورت پویا با گرفتن مقدار یک ورودی متنی اختصاص داده شود یا می‌توانید یک مقدار در اینجا اختصاص دهید. اگر این اولین بار است که با API تکمیل چت کار می‌کنید، باید `role` و `content` پیام را تعریف کنیم.

`role` می‌تواند یکی از `system` (ایجاد قوانین)، `assistant` (مدل) یا `user` (کاربر نهایی) باشد. برای فراخوانی تابع، ما این را به عنوان `user` و یک سوال نمونه اختصاص خواهیم داد.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### ایجاد توابع.

در ادامه یک تابع و پارامترهای آن تابع را تعریف خواهیم کرد. ما در اینجا فقط از یک تابع به نام `search_courses` استفاده خواهیم کرد اما شما می‌توانید چندین تابع ایجاد کنید.

**مهم**: توابع در پیام سیستمی به LLM گنجانده می‌شوند و در تعداد توکن‌های موجود شما محاسبه می‌شوند.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**تعاریف**

ساختار تعریف تابع چندین سطح دارد که هر کدام ویژگی‌های خاص خود را دارند. در اینجا یک تجزیه و تحلیل از ساختار تو در تو ارائه شده است:

**ویژگی‌های تابع در سطح بالا:**

`name` - نام تابعی که می‌خواهیم فراخوانی شود.

`description` - این توضیحی است درباره نحوه عملکرد تابع. در اینجا مهم است که دقیق و واضح باشید.

`parameters` - فهرستی از مقادیر و قالبی که می‌خواهید مدل در پاسخ خود تولید کند.

**ویژگی‌های شیء پارامترها:**

`type` - نوع داده شیء پارامترها (معمولاً "object")

`properties` - فهرستی از مقادیر خاصی که مدل برای پاسخ خود استفاده خواهد کرد.

**ویژگی‌های هر پارامتر جداگانه:**

`name` - به طور ضمنی توسط کلید ویژگی تعریف شده است (مثلاً "role"، "product"، "level")

`type` - نوع داده این پارامتر خاص (مثلاً "string"، "number"، "boolean")

`description` - توضیح پارامتر خاص.

**ویژگی‌های اختیاری:**

`required` - آرایه‌ای که پارامترهای مورد نیاز برای تکمیل فراخوانی تابع را فهرست می‌کند.


### فراخوانی تابع  
پس از تعریف یک تابع، اکنون باید آن را در فراخوانی API تکمیل چت وارد کنیم. این کار را با افزودن `functions` به درخواست انجام می‌دهیم. در این حالت `functions=functions` است.  

همچنین گزینه‌ای برای تنظیم `function_call` به `auto` وجود دارد. این بدان معناست که اجازه می‌دهیم مدل زبان بزرگ تصمیم بگیرد کدام تابع باید بر اساس پیام کاربر فراخوانی شود، به جای اینکه خودمان آن را تعیین کنیم.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

حالا بیایید به پاسخ نگاه کنیم و ببینیم چگونه قالب‌بندی شده است:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

می‌توانید ببینید که نام تابع فراخوانی شده است و از پیام کاربر، مدل زبان بزرگ توانسته داده‌ها را برای مطابقت با آرگومان‌های تابع پیدا کند.


## 3.ادغام فراخوانی‌های تابع در یک برنامه.

پس از اینکه پاسخ قالب‌بندی شده از LLM را آزمایش کردیم، اکنون می‌توانیم این را در یک برنامه ادغام کنیم.

### مدیریت جریان

برای ادغام این در برنامه خود، بیایید مراحل زیر را انجام دهیم:

ابتدا، بیایید فراخوانی به سرویس‌های OpenAI را انجام داده و پیام را در متغیری به نام `response_message` ذخیره کنیم.


In [ ]:
response_message = response.choices[0].message

اکنون تابعی را تعریف خواهیم کرد که API مایکروسافت لرن را برای دریافت فهرستی از دوره‌ها فراخوانی می‌کند:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



به عنوان یک بهترین روش، سپس بررسی می‌کنیم که آیا مدل می‌خواهد یک تابع را فراخوانی کند یا خیر. پس از آن، یکی از توابع موجود را ایجاد کرده و آن را با تابعی که فراخوانی می‌شود مطابقت می‌دهیم.  
سپس آرگومان‌های تابع را گرفته و آن‌ها را به آرگومان‌های LLM نگاشت می‌کنیم.

در نهایت، پیام فراخوانی تابع و مقادیری که توسط پیام `search_courses` بازگردانده شده‌اند را اضافه می‌کنیم. این به LLM تمام اطلاعات لازم را می‌دهد تا به کاربر با استفاده از زبان طبیعی پاسخ دهد.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



اکنون پیام به‌روزشده را به مدل زبان بزرگ (LLM) ارسال می‌کنیم تا بتوانیم پاسخ به زبان طبیعی دریافت کنیم به جای پاسخ قالب‌بندی شده JSON از API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## چالش کد

کار عالی! برای ادامه یادگیری خود در مورد فراخوانی تابع OpenAI می‌توانید بسازید: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
- پارامترهای بیشتری از تابع که ممکن است به یادگیرندگان کمک کند دوره‌های بیشتری پیدا کنند. می‌توانید پارامترهای API موجود را اینجا پیدا کنید:  
- ایجاد یک فراخوانی تابع دیگر که اطلاعات بیشتری از یادگیرنده مانند زبان مادری آن‌ها بگیرد  
- ایجاد مدیریت خطا زمانی که فراخوانی تابع و/یا فراخوانی API هیچ دوره مناسبی را باز نمی‌گرداند


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما در تلاش برای دقت هستیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است حاوی خطاها یا نواقصی باشند. سند اصلی به زبان بومی خود باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچ گونه سوءتفاهم یا تفسیر نادرستی که از استفاده این ترجمه ناشی شود، نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
